<a href="https://colab.research.google.com/github/345bc/DoAnNhom4-HeQuanTriCSDL/blob/main/2001230857-NguyenTanTuan-QuanTriTaiKhoan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

3.3.	Nhom_QuanTriTaiKhoan
3.3.1.	Mô tả ứng dụng
Ứng dụng được phát triển là hệ thống quản lý tài khoản người dùng được xây dựng trên nền tảng ASP.NET và SQL Server. Hệ thống cung cấp các chức năng chính bao gồm: đăng nhập, đăng ký, đăng xuất, khôi phục mật khẩu, cập nhật hồ sơ cá nhân và thay đổi mật khẩu. Đối với người dùng có vai trò quản trị viên (Admin), hệ thống cho phép xem danh sách toàn bộ tài khoản, thêm mới và chỉnh sửa thông tin tài khoản. Ngoài ra, ứng dụng còn tích hợp tính năng ghi nhật ký thay đổi mật khẩu (Audit Logs) nhằm theo dõi các hoạt động và nâng cao tính bảo mật. Về mặt công nghệ, ứng dụng sử dụng ASP.NET kết hợp với Entity Framework để thực hiện kết nối cơ sở dữ liệu, các thao tác CRUD (Create, Read, Update, Delete) và xác thực người dùng. Toàn bộ quy trình từ đăng nhập, quản lý tài khoản đến phân quyền người dùng đều đã được triển khai và kiểm thử đầy đủ.
3.3.2.	Các đối tượng xử lý và điều khiển:
[dbo].[sp_ChangeUserPassword]
CREATE PROCEDURE sp_ChangeUserPassword
    @UserID INT,
    @NewPasswordHash NVARCHAR(255),
    @Result INT OUTPUT              
AS
BEGIN
    SET NOCOUNT ON;
    
    BEGIN TRY
        UPDATE Users
        SET Password = @NewPasswordHash
        WHERE UserID = @UserID;

        IF EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[dbo].[AuditLog]'))
        BEGIN
            DECLARE @UserEmail NVARCHAR(100);
            SELECT @UserEmail = Email FROM Users WHERE UserID = @UserID;

            INSERT INTO AuditLog (ActionName, Description, PerformedBy, TargetUserEmail, ActionTime)
            VALUES (
                'CHANGE_PASSWORD',
                N'Người dùng tự thay đổi mật khẩu cá nhân',
                @UserEmail,
                @UserEmail,
                GETDATE()
            );
        END

        SET @Result = 0;
    END TRY
    BEGIN CATCH
        SET @Result = -1;
        
    END CATCH
END
-	Chức năng: sp_ChangeUserPassword là Stored Procedure dùng để thay đổi mật khẩu người dùng. Bao gồm: Cập nhật mật khẩu mới vào bảng Users Và Ghi lại lịch sử thay đổi mật khẩu vào bảng AuditLog.
Lưu thông tin gồm: hành động, mô tả, người thực hiện, email đích và thời gian.
Thủ tục trả về kết quả qua biến OUTPUT:
●	0 → Thành công
●	-1 → Lỗi trong quá trình xử lý
-	Cách hoạt động:
Bước 1: Kiểm tra tham số
Nhận vào:
●	@UserID → ID của người muốn đổi mật khẩu
●	@NewPasswordHash → mật khẩu mới đã được hash
●	@Result → biến OUTPUT báo kết quả
Bước 2: Bắt đầu TRY/CATCH
●	Đảm bảo khi xảy ra lỗi sẽ chuyển sang CATCH và trả về -1.
Bước 3: Cập nhật mật khẩu
UPDATE Users
      	SET Password = @NewPasswordHash
        	WHERE UserID = @UserID;
Bước 4: Kiểm tra xem bảng AuditLog có tồn tại
●	IF EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[dbo].[AuditLog]'))
Bước 5: Lấy email người dùng
●	SELECT @UserEmail = Email FROM Users WHERE UserID = @UserID;
Bước 6: Ghi lịch sử Audit Log
●	INSERT INTO AuditLog (...)
●	VALUES (...)
Bước 7: Trả kết quả
●	Nếu không có lỗi → @Result = 0
●	Nếu có lỗi → CATCH → @Result = -1
-	Kiểm thử:
Test 1 – Đổi mật khẩu hợp lệ:
DECLARE @res INT;
EXEC sp_ChangeUserPassword 1, 'hash_123', @res OUTPUT;
SELECT @res;

Output = 0, chạy thành công

Password của UserID = 1 thay đổi thành “hash_123”

Lưu 1 bản ghi mới vào AuditLog
Test 2 - UserID không tồn tại:
DECLARE @res INT;
EXEC sp_ChangeUserPassword 9999, '123', @res OUTPUT;
SELECT @res;

Output = 0, chạy thành công

Không có UserID = 999, không cập nhật bảng Users

AuditLog báo null
Test 3 – đổi mật khẩu từ ứng dụng

Đăng nhập tài khoản khách hàng và truy cập vào mục đổi mật khẩu, nhập mật khẩu hiện tại và xác nhận mật khẩu mới

Hệ thống xác nhận đổi mật khẩu thành công

Bảng user cập nhật mật khẩu mới dã Hash

Bảng AuditLog lưu 1 bản ghi mói
	[dbo].[sp_LoginUser]
CREATE PROCEDURE sp_LoginUser
		@Email NVARCHAR(100),
		@Result INT OUTPUT,        
		@UserID INT OUTPUT,
		@Name NVARCHAR(100) OUTPUT,
		@Role NVARCHAR(50) OUTPUT,
		@Password NVARCHAR(255) OUTPUT
	AS
	BEGIN
		SET NOCOUNT ON;

		SELECT
			@UserID = UserID,
			@Name = Name,
			@Role = Role,
			@Password = Password
		FROM Users
		WHERE Email = @Email;

		IF @UserID IS NULL
		BEGIN
			SET @Result = 1;
			RETURN;
		END

		SET @Result = 0;
	END
-	Chức năng: sp_LoginUser là Stored Procedure thực hiện kiểm tra thông tin đăng nhập dựa trên email người dùng. Bao gồm: Kiểm tra sự tồn tại của Email trong bảng Users. Nếu có, trả về các thông tin liên quan: UserID, Name, Role, Password (đã băm).
Ứng dụng C# sẽ tự đối chiếu mật khẩu người dùng nhập với mật khẩu hash trả về.
Trả kết quả qua biến OUTPUT:
●	0 → Tìm thấy user
●	1 → Không tìm thấy email
●	-1 → Lỗi
-	Cách hoạt động:
Bước 1: Nhận tham số đầu vào
●	@Email → email người dùng nhập.
Bước 2: Truy vấn bảng Users
SELECT
			@UserID = UserID,
			@Name = Name,
			@Role = Role,
			@Password = Password
		FROM Users
		WHERE Email = @Email;
Bước 3: Kiểm tra có tìm thấy User hay không
●	Nếu @UserID IS NULL → không tồn tại email → trả về @Result = 1.
Bước 4: Trả dữ liệu khi tìm thấy
●	Gán @Result = 0.
●	Trả về các thông tin:
UserID, Name, Role, Password (hash để C# verify).
-	Kiểm thử:
Test 1 – Email hợp lệ:
DECLARE @res INT, @id INT, @name NVARCHAR(100), @role NVARCHAR(50), @pass NVARCHAR(255);

EXEC sp_LoginUser
    'nguyentantuan2005bt@gmail.com',
    @res OUTPUT,
    @id OUTPUT,
    @name OUTPUT,
    @role OUTPUT,
    @pass OUTPUT;

SELECT @res AS Result, @id AS UserID, @name AS Name, @role AS Role, @pass AS PasswordHash;

Trả về kết quả Output = 0 tìm thấy User với UserID = 9 với Email là nguyentantuan2005bt@gmail.com
	Test 2 – Email không tồn tại
DECLARE @res INT, @id INT, @name NVARCHAR(100), @role NVARCHAR(50), @pass NVARCHAR(255);

EXEC sp_LoginUser
    'abc@xyz.com',
    @res OUTPUT,
    @id OUTPUT,
    @name OUTPUT,
    @role OUTPUT,
    @pass OUTPUT;

SELECT @res AS Result, @id AS UserID, @name AS Name, @role AS Role, @pass AS PasswordHash;

Trả về kết quả Output = 1 thể hiện Email không tồn tại
	Test 3 – Kiểm tra đăng nhập từ ứng dụng

Thông tin từ bảng Users

Nhập các thông tin Email, Password vào trong ô và nhấn nút Sign In

Chuyển hướng đến trang chủ của ứng dụng nếu nhập đúng thông tin

Báo lỗi khi nhập sai thông tin
	[dbo].[sp_RegisterUserWithSeparateName]
CREATE PROCEDURE sp_RegisterUserWithSeparateName
    @CustomerName NVARCHAR(100),    
    @UserName NVARCHAR(100),        
    @Email NVARCHAR(100),           
    @PasswordHash NVARCHAR(255),    
    @Role NVARCHAR(50) = 'Customer',
    @SoDT NVARCHAR(15) = NULL,
    @BirthDayKH DATETIME = NULL,
    @GioiTinhKH NVARCHAR(10) = NULL,
    
    @Result INT OUTPUT,    
    @UserID INT OUTPUT,     
    @MaKH INT OUTPUT        
AS
BEGIN
    SET NOCOUNT ON;

    IF EXISTS (SELECT 1 FROM Users WHERE Email = @Email OR Name = @UserName)
    BEGIN
        SET @Result = 1;
        SET @UserID = NULL;
        SET @MaKH = NULL;
        RETURN;
    END

    BEGIN TRY
        BEGIN TRANSACTION;

        INSERT INTO Users (Name, Email, Role, Password, CreatedAt)
        VALUES (@UserName, @Email, @Role, @PasswordHash, GETDATE());

        SET @UserID = SCOPE_IDENTITY();

        INSERT INTO KhachHang (TenKH, Email, SoDT, BirthDayKH, GioiTinhKH, UserId)
        VALUES (@CustomerName, @Email, @SoDT, @BirthDayKH, @GioiTinhKH, @UserID);

        SET @MaKH = SCOPE_IDENTITY();

        COMMIT TRANSACTION;
        SET @Result = 0;
    END TRY

    BEGIN CATCH
        IF @@TRANCOUNT > 0
            ROLLBACK TRANSACTION;

        SET @Result = -1;
        SET @UserID = NULL;
        SET @MaKH = NULL;
        
        THROW;
    END CATCH
END

-	Chức năng: sp_RegisterUserWithSeparateName là Stored Procedure dùng để đăng ký khách hàng mới, đồng thời tạo bản ghi trong cả hai bảng:
Users → chứa thông tin tài khoản đăng nhập
KhachHang → chứa thông tin hồ sơ cá nhân khách hàng
Thủ tục giúp đảm bảo dữ liệu đồng bộ giữa hai bảng và hỗ trợ hệ thống quản lý user + thông tin khách hàng.
-	Cách hoạt động:
Kiểm tra trùng lặp
Nếu Email hoặc UserName đã tồn tại → dừng lại, trả về @Result = 1.
Bắt đầu Transaction
INSERT vào bảng Users, lấy @UserID bằng SCOPE_IDENTITY()
INSERT vào bảng KhachHang, lấy @MaKH.
Commit Transaction nếu cả hai bảng đều thêm thành công.
Catch lỗi
Nếu xảy ra lỗi (như ràng buộc, kiểu dữ liệu, độ dài…)
→ rollback toàn bộ và trả về @Result = -1.
-	Kiểm thử:
Test 1 – Đăng ký hợp lệ
DECLARE @Out_Result INT;      
DECLARE @Out_UserID INT;      
DECLARE @Out_MaKH INT;        

EXEC sp_RegisterUserWithSeparateName
    @CustomerName = N'Nguyễn Văn A',
    @UserName     = N'user_test_02',
    @Email        = N'test001@email.com',
    @PasswordHash = N'matkhau123',
    @SoDT         = N'0901234567',
    @GioiTinhKH   = N'Nam',
    @Result       = @Out_Result OUTPUT,
    @UserID       = @Out_UserID OUTPUT,
    @MaKH         = @Out_MaKH OUTPUT;

SELECT
    N'Test 1' AS TenTest,
    @Out_Result AS KetQua_TraVe,
    CASE WHEN @Out_Result = 0 THEN N'✅ THÀNH CÔNG' ELSE N'❌ THẤT BẠI' END AS TrangThai,
    @Out_UserID AS New_UserID,
    @Out_MaKH AS New_MaKH;

Kết quả trả về 0 thể hiện đăng ký thành công

Bảng Users thêm 1 bản ghi mới

Bảng KhachHang lưu thông tin
	Test 2 – Email trùng trong bảng Users
DECLARE @Out_Result INT;      
DECLARE @Out_UserID INT;      
DECLARE @Out_MaKH INT;        

SET @Out_Result = NULL; SET @Out_UserID = NULL; SET @Out_MaKH = NULL;

EXEC sp_RegisterUserWithSeparateName
    @CustomerName = N'Nguyễn Văn B (Clone)',
    @UserName     = N'user_test_02_khac',
    @Email        = N'test01@email.com',  
    @PasswordHash = N'matkhau123',
    @Result       = @Out_Result OUTPUT,
    @UserID       = @Out_UserID OUTPUT,
    @MaKH         = @Out_MaKH OUTPUT;

SELECT
    N'Test 2 (Trùng Email)' AS TenTest,
    @Out_Result AS KetQua_TraVe,
    CASE WHEN @Out_Result = 1 THEN N'✅ CHẶN TỐT' ELSE N'❌ KHÔNG CHẶN ĐƯỢC' END AS TrangThai;
   
Kết quả trả về 1 Với lỗi trùng với email đã tồn tại
Lệnh ROLLBACK TRANSACTION; sẽ hủy toàn bộ các thao tác đã làm trong transaction
●	User không được tạo
●	KhachHang không được tạo
Test 3 – Đăng ký mới từ ứng dụng

Điền các thông tin vào ô nhập liệu và ấn Sign Up

Nếu nhập đúng hệ thống chuyển hướng sang trang đăng nhập đồng thời lưu thông tin vào bảng Users và bảng KhachHang

Lưu thông tin UserName, Email, Password, Role vào bảng Users

Lưu thông tin Full Name, Email, Phone Number, Birth Date, Gender vào bảng KhachHang

Nếu nhập không đúng thông tin hệ thống sẽ báo lỗi
	[dbo].[sp_ResetUserPassword]
CREATE PROCEDURE sp_ResetUserPassword
    @TargetEmail NVARCHAR(100),    
    @NewPassword NVARCHAR(255),    
    @AdminName NVARCHAR(100),      
    @Result INT OUTPUT             -- 0: Thành công, 1: Không tìm thấy Email, -1: Lỗi
AS
BEGIN
    SET NOCOUNT ON;
    
    IF NOT EXISTS (SELECT 1 FROM Users WHERE Email = @TargetEmail)
    BEGIN
        SET @Result = 1; -- Lỗi: Email không tồn tại
        RETURN;
    END

    BEGIN TRANSACTION;
    BEGIN TRY
        UPDATE Users
        SET Password = @NewPassword
        WHERE Email = @TargetEmail;

        INSERT INTO AuditLog (ActionName, Description, PerformedBy, TargetUserEmail)
        VALUES (
            'RESET_PASSWORD',
            N'Đã cấp lại mật khẩu mới cho tài khoản ' + @TargetEmail,
            @AdminName,
            @TargetEmail
        );

        COMMIT TRANSACTION;
        SET @Result = 0; -- Thành công
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION;
        SET @Result = -1; -- Lỗi hệ thống
        
    END CATCH
END
Chức năng: sp_ResetUserPassword là Stored Procedure dùng để cấp lại mật khẩu cho người dùng.
Các nhiệm vụ chính:
Kiểm tra email tồn tại trong bảng Users.
Cập nhật mật khẩu mới (đã băm từ phía C#).
Ghi nhật ký hành động (AuditLog).
Email người dùng
Trả kết quả qua biến OUTPUT:
●	0 → Thành công
●	1 → Không tìm thấy email
●	-1 → Lỗi hệ thống
-	Cách hoạt động:
Bước 1 – Kiểm tra Email
IF NOT EXISTS (SELECT 1 FROM Users WHERE Email = @TargetEmail)
●	Nếu email không tồn tại → @Result = 1, kết thúc SP.
Bước 2 – Bắt đầu giao dịch
BEGIN TRANSACTION;
●	Toàn bộ thao tác UPDATE + INSERT log nằm trong cùng một transaction.
Bước 3 – Cập nhật mật khẩu
        UPDATE Users
        SET Password = @NewPassword
        WHERE Email = @TargetEmail;
Bước 4 – Ghi log
INSERT INTO AuditLog (ActionName, Description, PerformedBy, TargetUserEmail)
        VALUES (
            'RESET_PASSWORD',
            N'Đã cấp lại mật khẩu mới cho tài khoản ' + @TargetEmail,
            @AdminName,
            @TargetEmail
        );
Bước 5 – Hoàn tất giao dịch
●	Nếu không lỗi → COMMIT TRANSACTION, @Result = 0
●	Nếu có lỗi → ROLLBACK TRANSACTION, @Result = -1
-	Kiểm thử:
Test 1 – Email hợp lệ
DECLARE @res INT;
EXEC sp_ResetUserPassword 'tuan@gmail.com', 'hash_newpass', 'Admin01', @res OUTPUT;
SELECT @res;

Kết quả trả về là 0 thể hiện đã thay đổi Password ở tài khoản 'tuan@gmail.com' về hasd_newpass

Password đã được thay đổi ở bảng Users

Lưu 1 bản ghi ở AuditLog
	Test 2 – Email không tồn tại
DECLARE @res INT;
EXEC sp_ResetUserPassword 'khongco@gmail.com', 'hash_newpass', 'Admin01', @res OUTPUT;
SELECT @res;

Kết quả trả về là 1 thể hiện không tìm thấy Email =>   Không thay đổi Users, Không thêm log
-	Test 3 – Thực hiện từ ứng dụng

Nhập Email vào ô nhập liệu để thay đổi mật khẩu

Báo lỗi nếu nhập sai

Nếu nhập đúng thông tin, kết hợp với hàm sinh mật khẩu ngẫu nhiên ở ứng dụng tạo mật khẩu mới cho tài khoản

Cập nhật Password mới ở bảng Users

Thêm bản ghi mới ở AuditLog
	[dbo].[fn_GetTotalCustomerCount]
CREATE FUNCTION fn_GetTotalCustomerCount()
RETURNS INT
AS
BEGIN
    DECLARE @CustomerCount INT;

    SELECT @CustomerCount = COUNT(MaKH)
    FROM KhachHang;

    RETURN @CustomerCount;
END;
-	Chức năng: fn_GetTotalCustomerCount là Function trả về tổng số khách hàng hiện có trong bảng KhachHang
-	Cách hoạt động:
●	Khai báo biến @CustomerCount để lưu số lượng khách hàng.
●	Dùng COUNT(MaKH) trên bảng KhachHang để đếm tổng số dòng (Mã khách hàng).
●	Gán kết quả vào @CustomerCount.
●	    RETURN @CustomerCount → trả về tổng số

-	Kiểm thử:
SELECT dbo.fn_GetTotalCustomerCount() AS TongSoKhachHang;

Trả về kết quả của Function

Hiển thị số lượng khách hàng ở Dashboard
	[tr_LimitAdminCount]
CREATE TRIGGER tr_LimitAdminCount
ON Users
AFTER INSERT
AS
BEGIN
    SET NOCOUNT ON;

    IF EXISTS (SELECT 1 FROM inserted WHERE Role = 'Admin')
    BEGIN
        DECLARE @CurrentAdminCount INT;
        
        SELECT @CurrentAdminCount = COUNT(UserID) FROM Users WHERE Role = 'Admin';
        
        IF @CurrentAdminCount > 2
        BEGIN
            ROLLBACK TRANSACTION;
            
            THROW 51000, N'Không thể tạo Admin mới. Hệ thống chỉ cho phép tối đa 2 quản trị viên (Admin).', 1;
            
            RETURN;
        END
    END
END
-	Chức năng: Trigger tr_LimitAdminCount dùng để giới hạn số lượng tài khoản Admin tối đa trong hệ thống.
Mỗi khi có bản ghi INSERT vào bảng Users, trigger sẽ kiểm tra xem tài khoản mới có phải Admin không.
Nếu tổng số Admin vượt quá 2, trigger sẽ:
•	Hủy bỏ thao tác INSERT
•	THROW lỗi để phía ứng dụng (C#, Entity Framework) nhận biết
Mục đích: Bảo vệ hệ thống, tránh việc tạo quá nhiều tài khoản quản trị.
-	Cách hoạt động:
Trigger chạy AFTER INSERT, tức là chỉ kích hoạt sau khi có lệnh INSERT vào bảng Users.
Kiểm tra dữ liệu trong bảng inserted (dữ liệu vừa được thêm).
IF EXISTS (SELECT 1 FROM inserted WHERE Role = 'Admin')
Đếm toàn bộ số Admin hiện có:
      	SELECT @CurrentAdminCount = COUNT(UserID) FROM Users WHERE Role = 'Admin';
Nếu số Admin > 2:
            ROLLBACK TRANSACTION → hủy việc thêm Admin
Gửi lỗi về ứng dụng:
            THROW 51000, N'Không thể tạo Admin mới. Hệ thống chỉ cho phép tối đa 2 quản trị viên (Admin).', 1;
Nhờ đó, không thể thêm quá 2 admin vào hệ thống.

-	Kiểm thử:
Test 1 – Chưa đủ 2 admin
INSERT INTO Users (Name, Email, Role, Password)
VALUES ('A', 'a@gmail.com', 'Admin', '123');

Thêm thành công Admin mới
	Test 2 – Vượt quá 2 admin
INSERT INTO Users (Name, Email, Role, Password)
VALUES ('C', 'c@gmail.com', 'Admin', '123');

Báo lỗi khi vượt quá 2 tài khoản Admin trong bảng Users
	Test 3 – Thêm mới từ ứng dụng

Danh sách tài khoản hiện có, đã tồn tại 2 tài khoản với role là Admin

Hệ thống báo lỗi ngay khi tạo mới
